# Implementation of Neural Style Transfer in Keras

**Import Libraries**

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import vgg19
from keras import backend as K
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

Using TensorFlow backend.


**Load and Preprocess Content Image and Style Image**

In [ ]:
content_image_path = 'content.jpg'
style_reference_image_path = 'style.jpg'

In [ ]:
width, height = load_img(content_image_path).size
img_height = 400
img_width = int(width * img_height / height)

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [ ]:
def deprocess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

**Load Pre-trained VGG19 Model**

In [ ]:
content_image = K.constant(preprocess_image(content_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

In [ ]:
input_tensor = K.concatenate([content_image, style_reference_image, combination_image], axis=0)

In [ ]:
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)
print('Model Loaded')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model Loaded


**Define Content Loss**

In [ ]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

**Define Gram Matrix**

In [ ]:
def gram_matrix(x):
    assert K.ndim(x) == 3
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

**Define Style Loss**

In [ ]:
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

**Define Total Variation Loss**

In [ ]:
def total_variational_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] -
        x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] -
        x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

**Compute Loss and Gradients**

In [ ]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [ ]:
content_layer = 'block5_conv2'
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

In [ ]:
total_variational_weight = 1e-4
style_weight = 1.
content_weight = 0.025

In [ ]:
loss = K.variable(0.)

In [ ]:
layer_features = outputs_dict[content_layer]
content_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]

In [ ]:
loss = loss + (content_weight * content_loss(base=content_image_features, combination=combination_features))

In [ ]:
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style=style_reference_features, combination=combination_features)
    loss += (style_weight / len(style_layers)) * sl

In [ ]:
loss = loss + (total_variational_weight * total_variational_loss(combination_image))

In [ ]:
grads = K.gradients(loss=loss, variables=combination_image)[0]

In [ ]:
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

In [ ]:
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grad_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape([1, img_height, img_width, 3])
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [ ]:
evaluator = Evaluator()

**Style Transfer Training**

In [ ]:
result_prefix = 'style_transfer_image'
iterations = 20

x = preprocess_image(content_image_path)
x = x.flatten()
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss,
                                     x,
                                     fprime=evaluator.grads,
                                     maxfun=20)
    print('Current loss value:', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    print('Image saved as', fname)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))


Start of iteration 0
Current loss value: 1424789100.0
Image saved as style_transfer_image_at_iteration_0.png
Iteration 0 completed in 461s
Start of iteration 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Current loss value: 658870600.0
Image saved as style_transfer_image_at_iteration_1.png
Iteration 1 completed in 457s
Start of iteration 2
Current loss value: 448784770.0
Image saved as style_transfer_image_at_iteration_2.png
Iteration 2 completed in 460s
Start of iteration 3
Current loss value: 338069020.0
Image saved as style_transfer_image_at_iteration_3.png
Iteration 3 completed in 458s
Start of iteration 4
Current loss value: 274188930.0
Image saved as style_transfer_image_at_iteration_4.png
Iteration 4 completed in 459s
Start of iteration 5
Current loss value: 242942850.0
Image saved as style_transfer_image_at_iteration_5.png
Iteration 5 completed in 457s
Start of iteration 6
Current loss value: 217116220.0
Image saved as style_transfer_image_at_iteration_6.png
Iteration 6 completed in 460s
Start of iteration 7
Current loss value: 197866800.0
Image saved as style_transfer_image_at_iteration_7.png
Iteration 7 completed in 461s
Start of iteration 8
Current loss value: 180948700.0
Im